In [1]:
#pip install pytorch

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [2]:
import torch
import numpy as np
from torch import nn

In [3]:
class rede_neural(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential (
             nn.Linear(1, 1)
        )

    def forward(self, x):
      return self.layers(x)

In [4]:
from torch.utils.data import Dataset, DataLoader
import torch.distributions.uniform as urand

class AlgebraicDataset(Dataset):
    def __init__(self, f, interval, nsamples):
      x = urand.Uniform(interval[0], interval[1]).sample([nsamples])
      self.data = [(x, ) for x in x ]

    def __len__(self):
      return len(self.data)

    def __getitem__(self, idx):
      return self.data[idx]

In [5]:
line = lambda x: 2*x + 3
interval = (-10, 10)
train_nsamples = 1000
test_nsamples = 100

In [6]:
train_dataset = AlgebraicDataset(line, interval, train_nsamples)
test_dataset = AlgebraicDataset(line, interval, test_nsamples)

train_dataloader = DataLoader(train_dataset, batch_size=train_nsamples, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_nsamples, shuffle=True)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Rodando na {device}" )

Rodando na cpu


In [8]:
model = rede_neural().to(device)

In [9]:
lossfunc = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(model, dataloader, lossfunc, optimizer):
  model.train()
  cumloss = 0.0
  for x, y in dataloader:
    x = x.unsqueeze().float().to(device)
    y = y.unsqueeze().float().to(device)

    pred = model (x)
    loss = lossfunc (pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cumloss += loss.item()

  return cumloss  / len(dataloader)

In [11]:
def test (model, dataloader, lossfunc,):
  model.eval()

  cumloss = 0.0
  with torch.no_grad():
    for x, y in dataloader:
      x = x.unsqueeze(0).float().to(device)
      y = y.unsqueeze(1).float().to(device)

      pred = model (x)
      loss = lossfunc (pred, y)

      cumloss += loss.item()

  return cumloss  / len(dataloader)

In [12]:
import matplotlib.pyplot as plt

In [13]:
def plot_comparinson(f, model, interval=(-10, 10), nsamples=10):
  fig, ax = plt.subplots(figsize=(-10, 10))

  ax.grid(True, which='both')
  ax.spines['left'].set_position('zero')
  ax.spines['right'].set_color('none')
  ax.spines['bottom'].set_position('zero')
  ax.spines['top'].set_color('none')

  samples = np.linspace(interval[0], interval[1], nsamples)
  model.eval()
  with torch.no.grad():
       pred = model(torch.tensor(samples).unsqueeze(1).float().to(device))


  ax.plot(samples, list(map(f, samples)), "o", label="ground truth")
  ax.plot(samples, pred.cpu(), label="model")
  plt.legend()
  plt.show()

In [18]:
epochs = 101
for t in range(epochs):
  train_loss = train(model, train_dataloader, lossfunc, optimizer)
  if t % 10 == 0:
    print(f"Epoch: {t}; Train Loss:{train_loss}")
    plot_comparinson(line, model)

test_loss = test (model, test_dataloader, lossfunc)
print(f"Test Loss: {test_loss}")

ValueError: ignored